<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/stack_fetch_california_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings;warnings.simplefilter('ignore')
from sklearn.datasets import load_iris,load_breast_cancer,fetch_california_housing,load_boston,load_diabetes,load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,auc,r2_score
from sklearn.ensemble import ExtraTreesClassifier,ExtraTreesRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier,XGBRegressor
from vecstack import stacking
import numpy as np
from sklearn.svm import SVC,SVR
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 載入資料

In [3]:
dataset = fetch_california_housing()
X = pd.DataFrame(dataset.data)
y = pd.Series(dataset.target,name='target')
full = X.join(y)
full.head()

,0,1,2,3,4,5,6,7,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


# 特徵縮放

In [4]:
from sklearn.preprocessing import StandardScaler
features = full.drop('target',axis=1).columns.tolist()
full[features] = StandardScaler().fit_transform(full[features])
full.head()

,0,1,2,3,4,5,6,7,target
0,2.344766,0.982143,0.628559,-0.153758,-0.974429,-0.049597,1.052548,-1.327835,4.526
1,2.332238,-0.607019,0.327041,-0.263336,0.861439,-0.092512,1.043185,-1.322844,3.585
2,1.782699,1.856182,1.155620,-0.049016,-0.820777,-0.025843,1.038503,-1.332827,3.521
3,0.932968,1.856182,0.156966,-0.049833,-0.766028,-0.050329,1.038503,-1.337818,3.413
4,-0.012881,1.856182,0.344711,-0.032906,-0.759847,-0.085616,1.038503,-1.337818,3.422


# train test split

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(full[features],full['target'],random_state=42)

# 空白結果表

In [0]:
result = pd.DataFrame()

# 簡單modeling

In [0]:
model = SVR().fit(X_train,y_train)
y_pred = model.predict(X_test)
result.loc['SVM','R2'] = r2_score(y_test, y_pred)

In [8]:
model = XGBRegressor().fit(X_train,y_train)
y_pred = model.predict(X_test)
result.loc['XGB','R2'] = r2_score(y_test, y_pred)

[07:58:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [0]:
model = KNeighborsRegressor().fit(X_train,y_train)
y_pred = model.predict(X_test)
result.loc['KNN','R2'] = r2_score(y_test, y_pred)

In [10]:
result

,R2
SVM,0.736951
XGB,0.781334
KNN,0.675202


# stacking

base models

In [11]:
# base model 1
model_1 = [SVR()]
oof_train_1, oof_test_1 = stacking(model_1,X_train, y_train, X_test,regression = True,metric = r2_score,n_folds = 5)
# base model 2
model_2 = [KNeighborsRegressor()]
oof_train_2, oof_test_2 = stacking(model_2,X_train, y_train, X_test,regression = True,metric = r2_score,n_folds = 5)
# base model 3
model_3 = [XGBRegressor()]
oof_train_3, oof_test_3 = stacking(model_3,X_train, y_train, X_test,regression = True,metric = r2_score,n_folds = 5)

[07:59:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:59:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:59:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:59:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:59:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


merge oof_train oof_test

In [12]:
oof_train = np.hstack([oof_train_1,oof_train_2,oof_train_3])
oof_test = np.hstack([oof_test_1,oof_test_2,oof_test_3])
print(oof_train.shape,oof_test.shape)

(15480, 3) (5160, 3)


level2 model

In [13]:
meta_model = XGBRegressor().fit(oof_train, y_train)
y_pred = meta_model.predict(oof_test)
result.loc['stack','R2'] = r2_score(y_test, y_pred)

[07:59:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [14]:
result.index.name = 'fetch_california_housing'
result

,R2
fetch_california_housing,
SVM,0.736951
XGB,0.781334
KNN,0.675202
stack,0.791766
